<a href="https://colab.research.google.com/github/Praharshita1275/Criminal_mind_analysis/blob/main/Criminal_mind_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# path = "https://drive.google.com/drive/folders/1KPv1lCH1Pol7HikFF7Qi8S5G3JbTACd3?usp=drive_link


#viraj


## llm1


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
#load dataset

df = pd.read_csv("/content/Crime_Data_from_2020_to_Present.csv")

df.head()
df.info()
df.isna().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326977 entries, 0 to 326976
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   DATE OCC      326977 non-null  object 
 1   TIME OCC      326977 non-null  int64  
 2   AREA NAME     326977 non-null  object 
 3   Vict Age      326977 non-null  int64  
 4   Vict Sex      326977 non-null  object 
 5   Vict Descent  326977 non-null  object 
 6   Premis Desc   326977 non-null  object 
 7   Weapon Desc   326977 non-null  object 
 8   Status Desc   326977 non-null  object 
 9   LOCATION      326977 non-null  object 
 10  LAT           326977 non-null  float64
 11  LON           326977 non-null  float64
 12  Crm Cd Desc   326977 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 32.4+ MB


,0
DATE OCC,0
TIME OCC,0
AREA NAME,0
Vict Age,0
Vict Sex,0
Vict Descent,0
Premis Desc,0
Weapon Desc,0
Status Desc,0
LOCATION,0


In [ ]:

#standardise column names

df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^a-z0-9_]", "", regex=True)
)

df.columns


Index(['date_occ', 'time_occ', 'area_name', 'vict_age', 'vict_sex',
       'vict_descent', 'premis_desc', 'weapon_desc', 'status_desc', 'location',
       'lat', 'lon', 'crm_cd_desc'],
      dtype='object')

In [ ]:
#HANDLE MISSING & INVALID VALUES
text_cols = [
    "area_name", "premis_desc", "weapon_desc",
    "status_desc", "crm_cd_desc", "location"
]

for col in text_cols:
    df[col] = df[col].fillna("UNKNOWN")

df["vict_age"] = df["vict_age"].replace(0, np.nan)
df["vict_age"] = df["vict_age"].fillna("UNKNOWN")



In [ ]:
# BASIC TEXT CLEANING (LIGHT)
def clean_text(col):
    return (
        col.astype(str)
        .str.lower()
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )

for col in text_cols:
    df[col] = clean_text(df[col])


In [ ]:
# FIX DATE & TIME
df["date_occ"] = pd.to_datetime(df["date_occ"], errors="coerce")

def time_to_hour(x):
    try:
        x = int(x)
        return x // 100
    except:
        return "UNKNOWN"

df["time_hour"] = df["time_occ"].apply(time_to_hour)


/tmp/ipython-input-1132658626.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date_occ"] = pd.to_datetime(df["date_occ"], errors="coerce")


In [ ]:
# SELECT ONLY USEFUL COLUMNS
keep_cols = [
    "date_occ", "time_hour", "area_name",
    "vict_age", "vict_sex",
    "premis_desc", "weapon_desc",
    "status_desc", "crm_cd_desc",
    "location"
]

df = df[keep_cols]



In [ ]:
# CREATE THE MOST IMPORTANT COLUMN (crime_text)
""" important step """
def create_crime_text(row):
    return (
        f"On {row['date_occ'].date()} at {row['time_hour']} hours, "
        f"in {row['area_name']} area, a {row['vict_age']}-year-old "
        f"{row['vict_sex']} was involved in {row['crm_cd_desc']} "
        f"at {row['premis_desc']}. "
        f"Weapon used: {row['weapon_desc']}. "
        f"Case status: {row['status_desc']}."
    )

df["crime_text"] = df.apply(create_crime_text, axis=1)


In [ ]:
df["crime_text"].head(3)


,crime_text
0,"On 2020-05-10 at 22 hours, in central area, a ..."
1,"On 2020-12-02 at 22 hours, in 77th street area..."
2,"On 2020-05-01 at 23 hours, in 77th street area..."


In [ ]:
#ADD SIMPLE RULE-BASED MOTIVATION LABEL
def infer_motivation(crime):
    crime = crime.lower()
    if "robbery" in crime or "theft" in crime:
        return "financial"
    elif "intimate partner" in crime or "rape" in crime:
        return "emotional"
    elif "assault" in crime or "weapon" in crime:
        return "power"
    else:
        return "unknown"

df["initial_motivation"] = df["crm_cd_desc"].apply(infer_motivation)


In [ ]:

print(df["crime_text"][1])

On 2020-12-02 at 22 hours, in 77th street area, a 21.0-year-old M was involved in robbery at street. Weapon used: verbal threat. Case status: invest cont.


In [ ]:
# Save processed data as CSV
df.to_csv("processed_crime_data.csv", index=False)

# -------------------------------
# Prepare JSON data for LLM usage
# -------------------------------

llm_data = []

for _, row in df.iterrows():
    llm_data.append({
        "text": row["crime_text"],
        "area": row["area_name"],
        "crime_type": row["crm_cd_desc"],
        "weapon": row["weapon_desc"],
        "motivation_hint": row["initial_motivation"]
    })

# Save LLM-ready JSON file
import json

with open("crime_data_llm_ready.json", "w", encoding="utf-8") as f:
    json.dump(llm_data, f, indent=2, ensure_ascii=False)


In [ ]:
# =====================================================
# LLM-1 SUPERVISED TRAINING (MOTIVATION CLASSIFIER)
# =====================================================

import pandas as pd
import pickle

from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# =====================================================
# STEP 1: Load CSV
# =====================================================
df = pd.read_csv("/content/processed_crime_data (1).csv")

# Keep only required columns
df = df[["crime_text", "initial_motivation"]]

# Remove unknown motivations
df = df[df["initial_motivation"] != "unknown"]

# Drop missing values
df = df.dropna()

print("Training samples:", len(df))
print(df["initial_motivation"].value_counts())

# =====================================================
# STEP 2: Encode Crime Text
# =====================================================
embedder = SentenceTransformer("all-MiniLM-L6-v2")

X = embedder.encode(
    df["crime_text"].tolist(),
    convert_to_numpy=True
)

y = df["initial_motivation"].tolist()

# =====================================================
# STEP 3: Train-Test Split
# =====================================================
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# =====================================================
# STEP 4: Train Classifier
# =====================================================
clf = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)

clf.fit(X_train, y_train)

# =====================================================
# STEP 5: Evaluate LLM-1
# =====================================================
y_pred = clf.predict(X_test)

print("\n✅ LLM-1 PERFORMANCE\n")
print("Accuracy:", round(accuracy_score(y_test, y_pred), 3))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# =====================================================
# STEP 6: Save Trained Model
# =====================================================
with open("/content/llm1_model.pkl", "wb") as f:
    pickle.dump(
        {
            "embedder": embedder,
            "classifier": clf,
            "labels": clf.classes_
        },
        f
    )

print("\n✅ LLM-1 model saved as /content/llm1_model.pkl")


Training samples: 278155
initial_motivation
power        171537
emotional     63329
financial     43289
Name: count, dtype: int64


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


✅ LLM-1 PERFORMANCE

Accuracy: 1.0

Classification Report:

              precision    recall  f1-score   support

   emotional       1.00      1.00      1.00     12666
   financial       1.00      1.00      1.00      8658
       power       1.00      1.00      1.00     34307

    accuracy                           1.00     55631
   macro avg       1.00      1.00      1.00     55631
weighted avg       1.00      1.00      1.00     55631


✅ LLM-1 model saved as /content/llm1_model.pkl


In [ ]:
import pickle

# =====================================================
# LOAD TRAINED LLM-1
# =====================================================
with open("/content/llm1_model.pkl", "rb") as f:
    model = pickle.load(f)

embedder = model["embedder"]
classifier = model["classifier"]

# =====================================================
# SAMPLE NEW CRIME TEXT
# =====================================================
new_crime_text = """
On 2021-03-12 at 21 hours, in central area, a 34-year-old M was involved
in robbery at street. Weapon used: hand gun. Case status: invest cont.
"""

# =====================================================
# PREDICT MOTIVATION
# =====================================================
embedding = embedder.encode([new_crime_text])
prediction = classifier.predict(embedding)[0]
confidence = max(classifier.predict_proba(embedding)[0])

print("Predicted Motivation:", prediction)
print("Confidence:", round(confidence, 3))


Predicted Motivation: financial
Confidence: 1.0


## llm2


In [ ]:
import pandas as pd
import json

# ===============================
# STEP 1: Load Murder Motives Data
# ===============================
df = pd.read_csv("/content/Murder Motives.csv")

# ===============================
# STEP 2: Clean Column Names
# ===============================
df.columns = (
    df.columns.str.lower()
              .str.replace(" ", "_")
              .str.replace("/", "_")
)

# ===============================
# STEP 3: Verify Column Names
# ===============================
# Check if the necessary column exists
required_columns = [
    "gain",
    "property_dispute",
    "personal_vendetta_or_enemity",
    "love_affairs_sexual_relations",
    "dowry",
    "communalism",
    "casteism",
    "political_reasons",
    "terrorists_extremists",
    "other_causes"
]

# Print the actual column names in the DataFrame to debug
print("Actual column names in the DataFrame:", df.columns)

# Check if the required columns exist in the DataFrame
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    print(f"Warning: The following columns are missing in the dataset: {missing_cols}")
else:
    print("All required columns are present.")

# ===============================
# STEP 4: Remove Aggregate Rows
# ===============================
df = df[~df["state"].str.contains("TOTAL", na=False)]

# ===============================
# STEP 5: Identify Motivation Columns (LLM-2)
# ===============================
# Ensure the column names match with the cleaned names
motivation_cols = [
    "gain",
    "property_dispute",
    "personal_vendetta_or_enemity",
    "love_affairs_sexual_relations",  # This should match the column name exactly
    "dowry",
    "communalism",
    "casteism",
    "political_reasons",
    "terrorists_extremists",
    "other_causes"
]

# ===============================
# STEP 6: Convert Each Row to Motivation Profile
# ===============================
llm2_data = []

for _, row in df.iterrows():
    # Build motivation profile for each row
    motivation_profile = {
        col: int(row[col]) if not pd.isna(row[col]) else 0
        for col in motivation_cols if col in df.columns  # Ensure the column exists
    }

    llm2_data.append({
        "state": row["state"],
        "year": row["year"],
        "motivation_distribution": motivation_profile
    })

# ===============================
# STEP 7: Save LLM-2 Dataset
# ===============================
with open("llm2_motivation_dataset.json", "w") as f:
    json.dump(llm2_data, f, indent=2)

print("✅ LLM-2 Motivation Dataset Created")


Actual column names in the DataFrame: Index(['state', 'year', 'gain', 'property_dispute',
       'personal_vendetta_or_enemity', 'love_affairs__sexual_relations',
       'dowry', 'lunacy', 'witchcraft', 'communalism', 'casteism',
       'class_conflict', 'political_reasons', 'terrorists__extremists',
       'other_causes', 'total'],
      dtype='object')
✅ LLM-2 Motivation Dataset Created


In [ ]:
import json
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

# =====================================================
# STEP 1 — LOAD JSON
# =====================================================
with open("/content/llm2_motivation_dataset.json", "r") as f:
    data = json.load(f)

# =====================================================
# STEP 2 — FLATTEN DATA
# =====================================================
records = []

for entry in data:
    record = {
        "state": entry["state"],
        "year": entry["year"]
    }
    record.update(entry["motivation_distribution"])
    records.append(record)

df = pd.DataFrame(records)

# =====================================================
# STEP 3 — ENCODE STATE
# =====================================================
le = LabelEncoder()
df["state_encoded"] = le.fit_transform(df["state"])

# =====================================================
# STEP 4 — FEATURES & TARGETS
# =====================================================
X = df[["state_encoded", "year"]]

motivation_cols = [
    col for col in df.columns
    if col not in ["state", "year", "state_encoded"]
]

y = df[motivation_cols]

# =====================================================
# STEP 5 — TRAIN-TEST SPLIT
# =====================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# =====================================================
# STEP 6 — TRAIN LLM-2
# =====================================================
model = MultiOutputRegressor(
    RandomForestRegressor(
        n_estimators=200,
        random_state=42
    )
)

model.fit(X_train, y_train)

# =====================================================
# STEP 7 — EVALUATION
# =====================================================
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f"\n✅ LLM-2 Mean Absolute Error: {round(mae, 2)}")

# =====================================================
# STEP 8 — SAVE MODEL
# =====================================================
with open("/content/llm2_model.pkl", "wb") as f:
    pickle.dump(
        {
            "model": model,
            "state_encoder": le,
            "motivation_columns": motivation_cols
        },
        f
    )

print("✅ LLM-2 model saved at /content/llm2_model.pkl")



✅ LLM-2 Mean Absolute Error: 40.98
✅ LLM-2 model saved at /content/llm2_model.pkl


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## llm 3

In [ ]:
import pandas as pd
import json

# ===============================
# STEP 1: Load Chicago Crimes Data
# ===============================
df = pd.read_csv("/content/Chicago_Crimes_2022.csv")

# ===============================
# STEP 2: Standardize Column Names
# ===============================
df.columns = (
    df.columns.str.lower()
              .str.replace(" ", "_")
              .str.replace(r"[^a-z0-9_]", "", regex=True)
)

# ===============================
# STEP 3: Select Context-Relevant Columns (LLM-3)
# ===============================
llm3_cols = [
    "date",
    "primary_type",
    "description",
    "location_description",
    "arrest",
    "domestic",
    "beat",
    "district",
    "ward",
    "community_area",
    "year",
    "latitude",
    "longitude"
]

df_llm3 = df[llm3_cols].fillna("UNKNOWN")

# ===============================
# STEP 4: Create Context Narrative (Text for LLM)
# ===============================
def build_context_text(row):
    return (
        f"In {row['year']}, a {row['primary_type']} incident occurred at a "
        f"{row['location_description']} location. "
        f"Domestic case: {row['domestic']}. "
        f"Arrest made: {row['arrest']}. "
        f"District {row['district']}, Beat {row['beat']}."
    )

df_llm3["context_text"] = df_llm3.apply(build_context_text, axis=1)

# ===============================
# STEP 5: Convert to LLM-3 JSON Format
# ===============================
llm3_data = []

for _, row in df_llm3.iterrows():
    llm3_data.append({
        "context_text": row["context_text"],
        "crime_type": row["primary_type"],
        "location_type": row["location_description"],
        "domestic": row["domestic"],
        "arrest": row["arrest"],
        "district": row["district"],
        "year": row["year"]
    })

# ===============================
# STEP 6: Save LLM-3 Dataset
# ===============================
with open("llm3_background_context.json", "w") as f:
    json.dump(llm3_data, f, indent=2)

print("✅ LLM-3 Background & Context Dataset Created")


/tmp/ipython-input-2420390228.py:7: DtypeWarning: Columns (0,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/Chicago_Crimes_2022.csv")


✅ LLM-3 Background & Context Dataset Created


In [ ]:
import json
import pickle
import numpy as np

from sentence_transformers import SentenceTransformer
from sklearn.cluster import MiniBatchKMeans

# =====================================================
# STEP 1 — LOAD LLM-3 DATA
# =====================================================
with open("/content/llm3_background_context.json", "r") as f:
    data = json.load(f)

texts = [item["context_text"] for item in data]

# =====================================================
# STEP 2 — ENCODE CONTEXT TEXT
# =====================================================
embedder = SentenceTransformer("all-MiniLM-L6-v2")
X = embedder.encode(
    texts,
    convert_to_numpy=True,
    batch_size=64,
    show_progress_bar=True
)

# =====================================================
# STEP 3 — FAST CLUSTERING (NO SILHOUETTE)
# =====================================================
# Fixed k based on domain intuition
NUM_CLUSTERS = 5

kmeans = MiniBatchKMeans(
    n_clusters=NUM_CLUSTERS,
    random_state=42,
    batch_size=256
)

cluster_labels = kmeans.fit_predict(X)

# =====================================================
# STEP 4 — ATTACH CLUSTERS
# =====================================================
for i, item in enumerate(data):
    item["llm3_cluster"] = int(cluster_labels[i])

# =====================================================
# STEP 5 — SAVE MODEL & DATA
# =====================================================
with open("/content/llm3_model.pkl", "wb") as f:
    pickle.dump(
        {
            "embedder": embedder,
            "kmeans": kmeans
        },
        f
    )

with open("/content/llm3_clustered_data.json", "w") as f:
    json.dump(data, f, indent=2)

print("✅ LLM-3 training complete (FAST VERSION)")


Batches:   0%|          | 0/3084 [00:00<?, ?it/s]

✅ LLM-3 training complete (FAST VERSION)


# llm4

In [ ]:
#STEP 1 — LOAD ALL PRE-TRAINED MODELS

import pickle
import json
import numpy as np

# Load LLM-1
with open("/content/llm1_model.pkl", "rb") as f:
    llm1 = pickle.load(f)

# Load LLM-2
with open("/content/llm2_model.pkl", "rb") as f:
    llm2 = pickle.load(f)

# Load LLM-3
with open("/content/llm3_model.pkl", "rb") as f:
    llm3 = pickle.load(f)

print("✅ All LLM models loaded")


✅ All LLM models loaded


In [ ]:
#STEP 2 — LOAD LLM-2 MOTIVATION JSON
with open("/content/llm2_motivation_dataset.json", "r") as f:
    llm2_data = json.load(f)

# Convert motivation JSON to fast lookup
motivation_lookup = {
    (item["state"], int(item["year"])): sum(item["motivation_distribution"].values())
    for item in llm2_data
}

print("✅ Motivation lookup ready")


✅ Motivation lookup ready


In [ ]:
#STEP 3 — LOAD LLM-3 CLUSTERED DATA
with open("/content/llm3_clustered_data.json", "r") as f:
    llm3_data = json.load(f)


In [ ]:
#STEP 4 — BUILD LLM-4 FEATURE VECTOR
X = []
y = []

for item in llm3_data:
    year = int(item["year"])
    state = "ANDHRA PRADESH"   # update if multiple states

    motivation_score = motivation_lookup.get((state, year), 0)

    features = [
        item["llm3_cluster"],
        int(item["domestic"]),
        int(item["arrest"]),
        item["district"],
        motivation_score
    ]

    X.append(features)

    # Target logic (example)
    y.append(1 if not item["arrest"] else 0)

X = np.array(X)
y = np.array(y)

print("✅ LLM-4 dataset created")
